In [8]:
from utilities import * 
from IPython.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))
%load_ext autoreload
%autoreload 2

In [20]:
from donut import DonutModel
from PIL import Image
import torch


import argparse
import json
import pickle
import glob
import os
import re
from pathlib import Path

import numpy as np
import pandas as pd
import torch
from datasets import load_dataset
from PIL import Image
from tqdm import tqdm


torch.cuda.is_available()

True

In [3]:
pretrained_model = DonutModel.from_pretrained(("../result/train_nist/20230901_162511"))

if torch.cuda.is_available():
    pretrained_model.half()
    pretrained_model.to("cuda")

pretrained_model.eval()


/home/sohrab/anaconda3/envs/ocr37/lib/python3.7/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3190.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


DonutModel(
  (encoder): SwinEncoder(
    (model): SwinTransformer(
      (patch_embed): PatchEmbed(
        (proj): Conv2d(3, 128, kernel_size=(4, 4), stride=(4, 4))
        (norm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
      )
      (pos_drop): Dropout(p=0.0, inplace=False)
      (layers): Sequential(
        (0): BasicLayer(
          (blocks): Sequential(
            (0): SwinTransformerBlock(
              (norm1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
              (attn): WindowAttention(
                (qkv): Linear(in_features=128, out_features=384, bias=True)
                (attn_drop): Dropout(p=0.0, inplace=False)
                (proj): Linear(in_features=128, out_features=128, bias=True)
                (proj_drop): Dropout(p=0.0, inplace=False)
                (softmax): Softmax(dim=-1)
              )
              (drop_path): Identity()
              (norm2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
              (mlp): M

In [18]:
image = Image.open("../dataset/raw/nist/nist_data/f0751_28.png").convert("RGB")
output = pretrained_model.inference(image=image, prompt="<s_>")["predictions"][0]
output

{'DATE': '8/9/89', 'CITY': 'cincinnati', 'STATE': 'OH.', 'ZIP': '45242'}

In [17]:
ground_truth = {'DATE': '8/9/89', 'CITY': 'cincinnati', 'STATE': 'OH.', 'ZIP': '45242'}
file_name = 'f0751_28.png'

In [11]:
dict_ = dict_distance_key(ground_truth, output, file_name)

In [14]:
 (dict_['total_accuracy'],
         dict_['date_accuracy'],
         dict_['city_accuracy'],
         dict_['state_accuracy'],
         dict_['zip_accuracy']) = key_based_accuracy(dict_['DATE'], dict_['CITY'], dict_['STATE'], dict_['ZIP'],
                                                     dict_['DATE_LEN'], dict_['CITY_LEN'], dict_['STATE_LEN'], dict_['ZIP_LEN'])

In [15]:
dict_

{'image_name': 'f0751_28.png',
 'DATE': 0,
 'DATE_LEN': 6,
 'CITY': 0,
 'CITY_LEN': 10,
 'STATE': 0,
 'STATE_LEN': 3,
 'ZIP': 0,
 'ZIP_LEN': 5,
 'total_accuracy': 100.0,
 'date_accuracy': 100.0,
 'city_accuracy': 100.0,
 'state_accuracy': 100.0,
 'zip_accuracy': 100.0}

# Test:

In [29]:
dataset_name_or_path = '../dataset/training/nist_form'
dataset = load_dataset(dataset_name_or_path, split='train')

Resolving data files:   0%|          | 0/1401 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/200 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/501 [00:00<?, ?it/s]

Found cached dataset imagefolder (/home/sohrab/.cache/huggingface/datasets/imagefolder/nist_form-0f3ce0e2f7336743/0.0.0/37fbb85cc714a338bea574ac6c7d0b5be5aff46c1862c1989b20e0771199e93f)


In [31]:
result_list = []
for _, sample in tqdm(enumerate(dataset), total=len(dataset)):

        file_path = sample['image'].filename
        file_name = f"{file_path[file_path.rfind('/')+1: ]}"

        ground_truth = json.loads(sample["ground_truth"])['gt_parse']

        output = pretrained_model.inference(image=sample["image"], prompt="<s_>")[
            "predictions"][0]

        dict_ = dict_distance_key(ground_truth, output, file_name)
        dict_['ground_truth'] = ground_truth

        (dict_['total_accuracy'],
         dict_['date_accuracy'],
         dict_['city_accuracy'],
         dict_['state_accuracy'],
         dict_['zip_accuracy']) = key_based_accuracy(dict_['DATE'], dict_['CITY'], dict_['STATE'], dict_['ZIP'],
                                                     dict_['DATE_LEN'], dict_['CITY_LEN'], dict_['STATE_LEN'], dict_['ZIP_LEN'])

        result_list.append(dict_)

 51%|███████████████████████████████████████████████████████████████████████████████▉                                                                            | 717/1400 [04:03<03:52,  2.94it/s]


KeyboardInterrupt: 